In [6]:
from  datasets import load_dataset
squad_it_dataset = load_dataset("json", data_files = "SQuAD_it-train.json", field = "data")

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
})

In [8]:
squad_it_dataset["train"][0]

{'title': 'Terremoto del Sichuan del 2008',
 'paragraphs': [{'context': "Il terremoto del Sichuan del 2008 o il terremoto del Gran Sichuan, misurato a 8.0 Ms e 7.9 Mw, e si è verificato alle 02:28:01 PM China Standard Time all' epicentro (06:28:01 UTC) il 12 maggio nella provincia del Sichuan, ha ucciso 69.197 persone e lasciato 18.222 dispersi.",
   'qas': [{'answers': [{'answer_start': 29, 'text': '2008'}],
     'id': '56cdca7862d2951400fa6826',
     'question': 'In quale anno si è verificato il terremoto nel Sichuan?'},
    {'answers': [{'answer_start': 232, 'text': '69.197'}],
     'id': '56cdca7862d2951400fa6828',
     'question': 'Quante persone sono state uccise come risultato?'},
    {'answers': [{'answer_start': 29, 'text': '2008'}],
     'id': '56d4f9902ccc5a1400d833c0',
     'question': 'Quale anno ha avuto luogo il terremoto del Sichuan?'},
    {'answers': [{'answer_start': 78, 'text': '8.0 Ms e 7.9 Mw'}],
     'id': '56d4f9902ccc5a1400d833c1',
     'question': 'Che cosa ha

In [10]:
data_files = {"train": "SQuAD_it-train.json", "test": "SQuAD_it-test.json"}
squad_it_dataset = load_dataset("json", data_files = data_files, field = "data")
squad_it_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
    test: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 48
    })
})

In [11]:
url = "https://github.com/crux82/squad-it/raw/master/"
data_files = {
    "train": url + "SQuAD_it-train.json.gz",
    "test": url + "SQuAD_it-test.json.gz",
}

In [35]:
from datasets import load_dataset
data_files = {"train": "drugsComTrain_raw.tsv", "test": "drugsComTest_raw.tsv"}
drugs_dataset = load_dataset("csv", data_files = data_files, delimiter = "\t")

In [36]:
drug_sample = drugs_dataset["train"].shuffle(seed = 1337).select(range(1000))
drug_sample[:3]

{'Unnamed: 0': [140993, 96435, 98979],
 'drugName': ['Escitalopram', 'Sertraline', 'Lutera'],
 'condition': ['Anxiety', 'Depression', 'Abnormal Uterine Bleeding'],
 'review': ['"Great drug - I was bed bound for months with crippling anxiety. I&#039;m out the house, I&#039;m back at work. I&#039;ve met the love of my life and I can&#039;t thank the psychiatrist who prescribed me this with quetiapine (serequel) 150mg at night. Don&#039;t just rely on the nhs for your mental health needs. I was lucky enough for my parents to send me to a private practioner who prescribed this combination and it genuinely saved my life. (I am based in the UK) (I am taking 20mg daily)"',
  '"I was put on Zoloft for Major Depression and General Anxiety Disorder after my baby was born about a year ago.  In general, it helped with my mood swings not being so intense, but I also found, over time, that it did not help me in stressful situations and I actually was more aggressive while taking Zoloft.  I have also

In [37]:
for split in drugs_dataset.keys():
    assert len(drugs_dataset[split]) == len(drugs_dataset[split].unique("Unnamed: 0"))

In [38]:
drugs_dataset = drugs_dataset.rename_column(original_column_name = "Unnamed: 0", new_column_name = "Patient_id")
drugs_dataset

DatasetDict({
    train: Dataset({
        features: ['Patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['Patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [39]:
print(len(drugs_dataset["train"].unique("drugName")), len(drugs_dataset["test"].unique("drugName")))
print(len(drugs_dataset["train"].unique("condition")), len(drugs_dataset["test"].unique("condition")))

3436 2637
885 709


In [40]:
def lowercase_condition(example: dict) -> dict:
    return {"condition": example["condition"].lower()}

drugs_dataset = drugs_dataset.map(lowercase_condition)

Map:   0%|          | 0/161297 [00:00<?, ? examples/s]

AttributeError: 'NoneType' object has no attribute 'lower'

In [41]:
drugs_dataset = drugs_dataset.filter(lambda x: x["condition"] is not None)

In [42]:
drugs_dataset = drugs_dataset.map(lowercase_condition)

In [43]:
drugs_dataset["train"]["condition"][:3]

['left ventricular dysfunction', 'adhd', 'birth control']

In [44]:
def compute_review_length(example: dict) -> dict:
    return {"review length": len(example["review"].split())}

In [46]:
drugs_dataset = drugs_dataset.map(compute_review_length)
drugs_dataset["train"][0]

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

{'Patient_id': 206461,
 'drugName': 'Valsartan',
 'condition': 'left ventricular dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'review length': 17}

In [47]:
drugs_dataset["train"].sort("review length")[:3]

{'Patient_id': [111469, 13653, 53602],
 'drugName': ['Ledipasvir / sofosbuvir',
  'Amphetamine / dextroamphetamine',
  'Alesse'],
 'condition': ['hepatitis c', 'adhd', 'birth control'],
 'review': ['"Headache"', '"Great"', '"Awesome"'],
 'rating': [10.0, 10.0, 10.0],
 'date': ['February 3, 2015', 'October 20, 2009', 'November 23, 2015'],
 'usefulCount': [41, 3, 0],
 'review length': [1, 1, 1]}

In [52]:
drugs_dataset = drugs_dataset.filter(lambda x: x["review length"] > 30)
drugs_dataset.num_rows

Filter:   0%|          | 0/138514 [00:00<?, ? examples/s]

Filter:   0%|          | 0/46108 [00:00<?, ? examples/s]

{'train': 138514, 'test': 46108}

In [57]:
drugs_dataset["train"].sort("review length")[-5:]

{'Patient_id': [165438, 79035, 216072, 181160, 121004],
 'drugName': ['Levonorgestrel',
  'Plan B One-Step',
  'Copper',
  'Prozac',
  'Venlafaxine'],
 'condition': ['emergency contraception',
  'emergency contraception',
  'birth control',
  'obsessive compulsive disorde',
  'migraine'],
 'review': ['"Okay ladies, I have decided to share my experience with Plan B One Step to hopefully help ease even just a few minds, because I know just how bad this situation sucks and how you are searching high and low for some honest feedback. No matter what I say here, you will still be freaking out for a while and overthinking every little thing you feel within your body. Just try your best to be positive and keep busy. \r\r\n\r\r\nAlright, so here goes the story: On Apr. 9 I had sex with a male partner and the condom broke. We didn&#039;t notice until the deed was done. Immediately I thought &quot;omg we are screwed&quot;. I had finished my last period on Apr. 4 (I was now dangerously close to ov

In [59]:
import html 

text = "I&#039;m a transformer called BERT"
html.unescape(text)

"I'm a transformer called BERT"

In [65]:
new_drugs_dataset = drugs_dataset.map(lambda x: {"review" : html.unescape(x["review"])})
new_drugs_dataset["train"]["review"][:5]

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

['"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. \r\nWe have tried many different medications and so far this is the most effective."',
 '"I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormone gestodene, which is not available in US, so I switched to Lybrel, because the ing

In [66]:
new_drugs_dataset = drugs_dataset.map(lambda x: {"review": [html.unescape(o) for o in x["review"]]}, batched = True)
new_drugs_dataset["train"]["review"][:5]

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

['"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. \r\nWe have tried many different medications and so far this is the most effective."',
 '"I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormone gestodene, which is not available in US, so I switched to Lybrel, because the ing

In [69]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(example):
    return tokenizer(example["review"], truncation = True)

In [70]:
tokenized_drugs_dataset = new_drugs_dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]